# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 12:36PM,241326,102,PBR-062422-04,"Senseonics, Incorporated",Released
1,Jul 7 2022 12:28PM,241325,10,MSP211934,"Methapharm, Inc.",Released
2,Jul 7 2022 12:28PM,241325,10,MSP211938,"Methapharm, Inc.",Released
3,Jul 7 2022 12:28PM,241325,10,MSP211939,"Methapharm, Inc.",Released
4,Jul 7 2022 12:28PM,241325,10,MSP211940,"Methapharm, Inc.",Released
5,Jul 7 2022 12:28PM,241325,10,MSP211941,"Methapharm, Inc.",Released
6,Jul 7 2022 12:28PM,241325,10,MSP211943,"Methapharm, Inc.",Released
7,Jul 7 2022 12:28PM,241325,10,MSP211944,"Methapharm, Inc.",Released
8,Jul 7 2022 12:28PM,241325,10,MSP211946,"Methapharm, Inc.",Released
9,Jul 7 2022 12:28PM,241325,10,MSP211949,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,241321,Released,2
51,241322,Released,1
52,241323,Released,16
53,241325,Released,13
54,241326,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241321,NaN,2.0
241322,NaN,1.0
241323,NaN,16.0
241325,NaN,13.0
241326,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241224,11.0,46.0
241228,0.0,1.0
241241,0.0,11.0
241243,0.0,5.0
241244,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241224,11,46
241228,0,1
241241,0,11
241243,0,5
241244,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241224,11,46
1,241228,0,1
2,241241,0,11
3,241243,0,5
4,241244,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241224,11,46
1,241228,,1
2,241241,,11
3,241243,,5
4,241244,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated"
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc."
14,Jul 7 2022 12:25PM,241323,10,Beach Products Inc
30,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc."
31,Jul 7 2022 12:17PM,241321,10,Emerginnova
33,Jul 7 2022 12:15PM,241320,102,"Senseonics, Incorporated"
34,Jul 7 2022 12:14PM,241319,20,"ACI Healthcare USA, Inc."
77,Jul 7 2022 12:05PM,241317,10,ISDIN Corporation
78,Jul 7 2022 12:02PM,241315,10,ISDIN Corporation
87,Jul 7 2022 12:02PM,241316,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated",,1
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc.",,13
2,Jul 7 2022 12:25PM,241323,10,Beach Products Inc,,16
3,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc.",,1
4,Jul 7 2022 12:17PM,241321,10,Emerginnova,,2
5,Jul 7 2022 12:15PM,241320,102,"Senseonics, Incorporated",,1
6,Jul 7 2022 12:14PM,241319,20,"ACI Healthcare USA, Inc.",,43
7,Jul 7 2022 12:05PM,241317,10,ISDIN Corporation,,1
8,Jul 7 2022 12:02PM,241315,10,ISDIN Corporation,,9
9,Jul 7 2022 12:02PM,241316,10,"Citieffe, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated",1,
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc.",13,
2,Jul 7 2022 12:25PM,241323,10,Beach Products Inc,16,
3,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc.",1,
4,Jul 7 2022 12:17PM,241321,10,Emerginnova,2,
5,Jul 7 2022 12:15PM,241320,102,"Senseonics, Incorporated",1,
6,Jul 7 2022 12:14PM,241319,20,"ACI Healthcare USA, Inc.",43,
7,Jul 7 2022 12:05PM,241317,10,ISDIN Corporation,1,
8,Jul 7 2022 12:02PM,241315,10,ISDIN Corporation,9,
9,Jul 7 2022 12:02PM,241316,10,"Citieffe, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated",1,
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc.",13,
2,Jul 7 2022 12:25PM,241323,10,Beach Products Inc,16,
3,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc.",1,
4,Jul 7 2022 12:17PM,241321,10,Emerginnova,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated",1.0,NaN
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc.",13.0,NaN
2,Jul 7 2022 12:25PM,241323,10,Beach Products Inc,16.0,NaN
3,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc.",1.0,NaN
4,Jul 7 2022 12:17PM,241321,10,Emerginnova,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 12:36PM,241326,102,"Senseonics, Incorporated",1.0,0.0
1,Jul 7 2022 12:28PM,241325,10,"Methapharm, Inc.",13.0,0.0
2,Jul 7 2022 12:25PM,241323,10,Beach Products Inc,16.0,0.0
3,Jul 7 2022 12:17PM,241322,21,"NBTY Global, Inc.",1.0,0.0
4,Jul 7 2022 12:17PM,241321,10,Emerginnova,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7238442,344.0,0.0
102,965249,4.0,0.0
15,482509,44.0,17.0
16,241256,1.0,0.0
19,965077,53.0,12.0
20,723883,66.0,2.0
21,1688807,7.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7238442,344.0,0.0
1,102,965249,4.0,0.0
2,15,482509,44.0,17.0
3,16,241256,1.0,0.0
4,19,965077,53.0,12.0
5,20,723883,66.0,2.0
6,21,1688807,7.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,344.0,0.0
1,102,4.0,0.0
2,15,44.0,17.0
3,16,1.0,0.0
4,19,53.0,12.0
5,20,66.0,2.0
6,21,7.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,344.0
1,102,Released,4.0
2,15,Released,44.0
3,16,Released,1.0
4,19,Released,53.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20,21
Status,,,,,,,
Executing,0.0,0.0,17.0,0.0,12.0,2.0,0.0
Released,344.0,4.0,44.0,1.0,53.0,66.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20,21
0,Executing,0.0,0.0,17.0,0.0,12.0,2.0,0.0
1,Released,344.0,4.0,44.0,1.0,53.0,66.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20,21
0,Executing,0.0,0.0,17.0,0.0,12.0,2.0,0.0
1,Released,344.0,4.0,44.0,1.0,53.0,66.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()